# ItaData2024

In [1]:
using Pkg
Pkg.activate(".")
using MLJ, ModalDecisionTrees
using SoleDecisionTreeInterface, Sole, SoleData
using CategoricalArrays
using DataFrames, JLD2, CSV
using Audio911
using Random
using StatsBase, Catch22
using Test
using Plots, Printf
;

  Activating project at `~/Documents/Aclai/audio-rules2024`


### Collect rules settings

In [2]:
struct DisplayRule
    rule::ClassificationRule{<:AbstractString}
    printrule::String
    consequent::String
    metrics::NamedTuple
    experiment::NamedTuple
end

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]

rules_dict = Dict(exp => DisplayRule[] for exp in avail_exp)

function collect_rules!(
    rules_dict::Dict{Symbol, Vector{DisplayRule}}, 
    interesting_rules::AbstractVector{<:ClassificationRule{<:AbstractString}}, 
    experiment::NamedTuple;
    variable_names::Union{Nothing, AbstractVector{<:AbstractString}}=nothing
)
    string_rules = [string(rule)[14:end-1] for rule in interesting_rules]
    antecedent, consequent = collect.(zip(split.(string_rules, "  ↣  ")...))
    metrics = readmetrics.(interesting_rules, round_digits=2)

    if !isnothing(variable_names)
        antecedent = replace.(antecedent, r"\[V(\d+)\]" => s -> "($(variable_names[parse(Int, s[3:end-1])][6:end-4]))")
    end

    printrules = replace.(antecedent, 
        r"\e\[1m(.*?) \e\[1m(.*?)\e\[0m\e\[0m" => s"\1 \2",
        r"\d+\.\d{3,}" => m -> @sprintf("%.2f", parse(Float64, m))
    )

    append!(rules_dict[experiment.condition], DisplayRule(interesting_rules[i], printrules[i], string(consequent[i]), metrics[i], experiment) for i in eachindex(interesting_rules))
end
;

## propositional analysis intersting rules comparision

In [3]:
sr = 8000

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment), y)

color_code = Dict(:red => 31, :green => 32, :yellow => 33, :blue => 34, :magenta => 35, :cyan => 36);
r_select = r"\e\[\d+m(.*?)\e\[0m";
catch9_f = ["max", "min", "mean", "med", "std", "bsm", "bsd", "qnt", "3ac"]
catch9 = [
    maximum,
    minimum,
    StatsBase.mean,
    median,
    std,
    Catch22.SB_BinaryStats_mean_longstretch1,
    Catch22.SB_BinaryStats_diff_longstretch0,
    Catch22.SB_MotifThree_quantile_hh,
    Catch22.SB_TransitionMatrix_3ac_sumdiagcov,
]

9-element Vector{Function}:
 maximum (generic function with 106 methods)
 minimum (generic function with 106 methods)
 mean (generic function with 117 methods)
 median (generic function with 54 methods)
 std (generic function with 19 methods)
 SuperFeature SB_BinaryStats_mean_longstretch1 with fields:
description: Longest period of consecutive values above the mean
   keywords: distribution, stationarity
 SuperFeature SB_BinaryStats_diff_longstretch0 with fields:
description: Longest period of successive incremental decreases
   keywords: distribution, stationarity
 SuperFeature SB_MotifThree_quantile_hh with fields:
description: Shannon entropy of two successive letters in equiprobable 3-letter symbolization
   keywords: symbolic, motifs
 SuperFeature SB_TransitionMatrix_3ac_sumdiagcov with fields:
description: Trace of covariance of transition matrix between symbols in 3-letter alphabet
   keywords: symbolic, transitionmat

### Pneumonia
scale: semitones without mfcc

In [4]:
experiment = (
    type = :propositional,
    # type = :modal,

    condition = :Pneumonia,
    # condition = :Bronchiectasis,
    # condition = :COPD,
    # condition = :URTI,
    # condition = :Bronchiolitis,

    scale = :semitones,
    # scale = :mel_htk,

    featset = (),
    # featset = (:mfcc,),
)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = experiment.scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = experiment.scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = experiment.scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in experiment.featset ? false : true,
    audio_norm = true,
)

@assert experiment.condition in avail_exp "Unknown type of experiment: $(experiment.condition)."

ds_path = "/datasets/respiratory_Healthy_" * String(experiment.condition)
filename = "/datasets/itadata2024_" * String(experiment.condition) * "_files"

destpath = "results/propositional/$(experiment.scale)"
:mfcc in experiment.featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in experiment.featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in experiment.featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
)

collect_rules!(rules_dict, interesting_rules, experiment)

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X10sZmlsZQ==.jl:59


4-element Vector{DisplayRule}:
 DisplayRule(▣ (std(skwns) < 16.8775664684418) ∧ (std(mel10=1687Hz) < 0.5352387834210879) ∧ (min(mel8=1194Hz) < -4.203758541572919) ∧ (std(mel8=1194Hz) < 0.18671195168672658)  ↣  Pneumonia
, "(std(skwns) < 16.88) ∧ (std(mel10=1687Hz) < 0.54) ∧ (min(mel8=1194Hz) < -4.20) ∧ (std(mel8=1194Hz) < 0.19)", "Pneumonia", (ninstances = 101, ncovered = 17, coverage = 0.17, confidence = 1.0, lift = 2.2, natoms = 4), (type = :propositional, condition = :Pneumonia, scale = :semitones, featset = ()))
 DisplayRule(▣ (std(skwns) < 16.8775664684418) ∧ (std(mel10=1687Hz) < 0.5352387834210879) ∧ (min(mel8=1194Hz) < -4.203758541572919) ∧ (std(mel8=1194Hz) ≥ 0.18671195168672658) ∧ (max(mel6=845Hz) ≥ -3.5792143086384973) ∧ (std(mel6=845Hz) ≥ 0.48144848701559795) ∧ (max(decrs) < 0.9335043307891185)  ↣  Pneumonia
, "(std(skwns) < 16.88) ∧ (std(mel10=1687Hz) < 0.54) ∧ (min(mel8=1194Hz) < -4.20) ∧ (std(mel8=1194Hz) ≥ 0.19) ∧ (max(mel6=845Hz) ≥ -3.58) ∧ (std(mel6=845Hz) ≥ 0.48) ∧ (m

### Pneumonia
scale: semitones with mfcc

In [5]:
experiment = (
    type = :propositional,
    # type = :modal,

    condition = :Pneumonia,
    # condition = :Bronchiectasis,
    # condition = :COPD,
    # condition = :URTI,
    # condition = :Bronchiolitis,

    scale = :semitones,
    # scale = :mel_htk,

    # featset = (),
    featset = (:mfcc,),
)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = experiment.scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = experiment.scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = experiment.scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in experiment.featset ? false : true,
    audio_norm = true,
)

@assert experiment.condition in avail_exp "Unknown type of experiment: $(experiment.condition)."

ds_path = "/datasets/respiratory_Healthy_" * String(experiment.condition)
filename = "/datasets/itadata2024_" * String(experiment.condition) * "_files"

destpath = "results/propositional/$(experiment.scale)"
:mfcc in experiment.featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in experiment.featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in experiment.featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
)

collect_rules!(rules_dict, interesting_rules, experiment)

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X12sZmlsZQ==.jl:59


6-element Vector{DisplayRule}:
 DisplayRule(▣ (std(skwns) < 16.8775664684418) ∧ (std(mel10=1687Hz) < 0.5352387834210879) ∧ (min(mel8=1194Hz) < -4.203758541572919) ∧ (std(mel8=1194Hz) < 0.18671195168672658)  ↣  Pneumonia
, "(std(skwns) < 16.88) ∧ (std(mel10=1687Hz) < 0.54) ∧ (min(mel8=1194Hz) < -4.20) ∧ (std(mel8=1194Hz) < 0.19)", "Pneumonia", (ninstances = 101, ncovered = 17, coverage = 0.17, confidence = 1.0, lift = 2.2, natoms = 4), (type = :propositional, condition = :Pneumonia, scale = :semitones, featset = ()))
 DisplayRule(▣ (std(skwns) < 16.8775664684418) ∧ (std(mel10=1687Hz) < 0.5352387834210879) ∧ (min(mel8=1194Hz) < -4.203758541572919) ∧ (std(mel8=1194Hz) ≥ 0.18671195168672658) ∧ (max(mel6=845Hz) ≥ -3.5792143086384973) ∧ (std(mel6=845Hz) ≥ 0.48144848701559795) ∧ (max(decrs) < 0.9335043307891185)  ↣  Pneumonia
, "(std(skwns) < 16.88) ∧ (std(mel10=1687Hz) < 0.54) ∧ (min(mel8=1194Hz) < -4.20) ∧ (std(mel8=1194Hz) ≥ 0.19) ∧ (max(mel6=845Hz) ≥ -3.58) ∧ (std(mel6=845Hz) ≥ 0.48) ∧ (m

### Bronchiectasis
scale: semitones without mfcc

In [6]:
experiment = (
    type = :propositional,
    # type = :modal,

    # condition = :Pneumonia,
    condition = :Bronchiectasis,
    # condition = :COPD,
    # condition = :URTI,
    # condition = :Bronchiolitis,

    scale = :semitones,
    # scale = :mel_htk,

    featset = (),
    # featset = (:mfcc,),
)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = experiment.scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = experiment.scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = experiment.scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in experiment.featset ? false : true,
    audio_norm = true,
)

@assert experiment.condition in avail_exp "Unknown type of experiment: $(experiment.condition)."

ds_path = "/datasets/respiratory_Healthy_" * String(experiment.condition)
filename = "/datasets/itadata2024_" * String(experiment.condition) * "_files"

destpath = "results/propositional/$(experiment.scale)"
:mfcc in experiment.featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in experiment.featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in experiment.featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
)

collect_rules!(rules_dict, interesting_rules, experiment)

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X14sZmlsZQ==.jl:59


2-element Vector{DisplayRule}:
 DisplayRule(▣ std(decrs) < 2.7746655481160873  ↣  Bronchiectasis
, "std(decrs) < 2.77", "Bronchiectasis", (ninstances = 36, ncovered = 18, coverage = 0.5, confidence = 1.0, lift = 2.0, natoms = 1), (type = :propositional, condition = :Bronchiectasis, scale = :semitones, featset = ()))
 DisplayRule(▣ std(decrs) ≥ 2.7746655481160873  ↣  Healthy
, "std(decrs) ≥ 2.77", "Healthy", (ninstances = 36, ncovered = 18, coverage = 0.5, confidence = 1.0, lift = 2.0, natoms = 1), (type = :propositional, condition = :Bronchiectasis, scale = :semitones, featset = ()))

### Bronchiectasis
scale: semitones with mfcc

In [7]:
experiment = (
    type = :propositional,
    # type = :modal,

    # condition = :Pneumonia,
    condition = :Bronchiectasis,
    # condition = :COPD,
    # condition = :URTI,
    # condition = :Bronchiolitis,

    scale = :semitones,
    # scale = :mel_htk,

    # featset = (),
    featset = (:mfcc,),
)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = experiment.scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = experiment.scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = experiment.scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in experiment.featset ? false : true,
    audio_norm = true,
)

@assert experiment.condition in avail_exp "Unknown type of experiment: $(experiment.condition)."

ds_path = "/datasets/respiratory_Healthy_" * String(experiment.condition)
filename = "/datasets/itadata2024_" * String(experiment.condition) * "_files"

destpath = "results/propositional/$(experiment.scale)"
:mfcc in experiment.featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in experiment.featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in experiment.featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
)

collect_rules!(rules_dict, interesting_rules, experiment)

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X16sZmlsZQ==.jl:59


4-element Vector{DisplayRule}:
 DisplayRule(▣ std(decrs) < 2.7746655481160873  ↣  Bronchiectasis
, "std(decrs) < 2.77", "Bronchiectasis", (ninstances = 36, ncovered = 18, coverage = 0.5, confidence = 1.0, lift = 2.0, natoms = 1), (type = :propositional, condition = :Bronchiectasis, scale = :semitones, featset = ()))
 DisplayRule(▣ std(decrs) ≥ 2.7746655481160873  ↣  Healthy
, "std(decrs) ≥ 2.77", "Healthy", (ninstances = 36, ncovered = 18, coverage = 0.5, confidence = 1.0, lift = 2.0, natoms = 1), (type = :propositional, condition = :Bronchiectasis, scale = :semitones, featset = ()))
 DisplayRule(▣ mean(decrs) < -1.4909694112089293  ↣  Healthy
, "mean(decrs) < -1.49", "Healthy", (ninstances = 36, ncovered = 22, coverage = 0.61, confidence = 1.0, lift = 1.64, natoms = 1), (type = :propositional, condition = :Bronchiectasis, scale = :semitones, featset = (:mfcc,)))
 DisplayRule(▣ mean(decrs) ≥ -1.4909694112089293  ↣  Bronchiectasis
, "mean(decrs) ≥ -1.49", "Bronchiectasis", (ninstances =

### COPD
scale: semitones without mfcc

In [8]:
experiment = (
    type = :propositional,
    # type = :modal,

    # condition = :Pneumonia,
    # condition = :Bronchiectasis,
    condition = :COPD,
    # condition = :URTI,
    # condition = :Bronchiolitis,

    scale = :semitones,
    # scale = :mel_htk,

    featset = (),
    # featset = (:mfcc,),
)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = experiment.scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = experiment.scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = experiment.scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in experiment.featset ? false : true,
    audio_norm = true,
)

@assert experiment.condition in avail_exp "Unknown type of experiment: $(experiment.condition)."

ds_path = "/datasets/respiratory_Healthy_" * String(experiment.condition)
filename = "/datasets/itadata2024_" * String(experiment.condition) * "_files"

destpath = "results/propositional/$(experiment.scale)"
:mfcc in experiment.featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in experiment.featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in experiment.featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
)

collect_rules!(rules_dict, interesting_rules, experiment)

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X21sZmlsZQ==.jl:59


2-element Vector{DisplayRule}:
 DisplayRule(▣ (med(entrp) < 0.20939694020224994) ∧ (min(mel12=2383Hz) ≥ -8.020823114106859) ∧ (std(mel10=1687Hz) ≥ 0.14575752636971573) ∧ (mean(slope) ≥ -0.03440939539928442)  ↣  Healthy
, "(med(entrp) < 0.21) ∧ (min(mel12=2383Hz) ≥ -8.02) ∧ (std(mel10=1687Hz) ≥ 0.15) ∧ (mean(slope) ≥ -0.03)", "Healthy", (ninstances = 110, ncovered = 26, coverage = 0.24, confidence = 0.96, lift = 1.86, natoms = 4), (type = :propositional, condition = :COPD, scale = :semitones, featset = ()))
 DisplayRule(▣ (med(entrp) ≥ 0.22865762667347816) ∧ (min(mel14=3366Hz) ≥ -5.871457993702041)  ↣  COPD
, "(med(entrp) ≥ 0.23) ∧ (min(mel14=3366Hz) ≥ -5.87)", "COPD", (ninstances = 110, ncovered = 30, coverage = 0.27, confidence = 0.97, lift = 2.01, natoms = 2), (type = :propositional, condition = :COPD, scale = :semitones, featset = ()))

### COPD
scale: semitones with mfcc

In [9]:
experiment = (
    type = :propositional,
    # type = :modal,

    # condition = :Pneumonia,
    # condition = :Bronchiectasis,
    condition = :COPD,
    # condition = :URTI,
    # condition = :Bronchiolitis,

    scale = :semitones,
    # scale = :mel_htk,

    # featset = (),
    featset = (:mfcc,),
)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = experiment.scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = experiment.scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = experiment.scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in experiment.featset ? false : true,
    audio_norm = true,
)

@assert experiment.condition in avail_exp "Unknown type of experiment: $(experiment.condition)."

ds_path = "/datasets/respiratory_Healthy_" * String(experiment.condition)
filename = "/datasets/itadata2024_" * String(experiment.condition) * "_files"

destpath = "results/propositional/$(experiment.scale)"
:mfcc in experiment.featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in experiment.featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in experiment.featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
)

collect_rules!(rules_dict, interesting_rules, experiment)

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X23sZmlsZQ==.jl:59


5-element Vector{DisplayRule}:
 DisplayRule(▣ (med(entrp) < 0.20939694020224994) ∧ (min(mel12=2383Hz) ≥ -8.020823114106859) ∧ (std(mel10=1687Hz) ≥ 0.14575752636971573) ∧ (mean(slope) ≥ -0.03440939539928442)  ↣  Healthy
, "(med(entrp) < 0.21) ∧ (min(mel12=2383Hz) ≥ -8.02) ∧ (std(mel10=1687Hz) ≥ 0.15) ∧ (mean(slope) ≥ -0.03)", "Healthy", (ninstances = 110, ncovered = 26, coverage = 0.24, confidence = 0.96, lift = 1.86, natoms = 4), (type = :propositional, condition = :COPD, scale = :semitones, featset = ()))
 DisplayRule(▣ (med(entrp) ≥ 0.22865762667347816) ∧ (min(mel14=3366Hz) ≥ -5.871457993702041)  ↣  COPD
, "(med(entrp) ≥ 0.23) ∧ (min(mel14=3366Hz) ≥ -5.87)", "COPD", (ninstances = 110, ncovered = 30, coverage = 0.27, confidence = 0.97, lift = 2.01, natoms = 2), (type = :propositional, condition = :COPD, scale = :semitones, featset = ()))
 DisplayRule(▣ (mean(crest) < 120.66469656425167) ∧ (qnt(mel6=845Hz) ≥ 1.7341708652861298)  ↣  COPD
, "(mean(crest) < 120.66) ∧ (qnt(mel6=845Hz) ≥ 1.

### URTI
scale: semitones without mfcc

In [10]:
experiment = (
    type = :propositional,
    # type = :modal,

    # condition = :Pneumonia,
    # condition = :Bronchiectasis,
    # condition = :COPD,
    condition = :URTI,
    # condition = :Bronchiolitis,

    scale = :semitones,
    # scale = :mel_htk,

    featset = (),
    # featset = (:mfcc,),
)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = experiment.scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = experiment.scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = experiment.scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in experiment.featset ? false : true,
    audio_norm = true,
)

@assert experiment.condition in avail_exp "Unknown type of experiment: $(experiment.condition)."

ds_path = "/datasets/respiratory_Healthy_" * String(experiment.condition)
filename = "/datasets/itadata2024_" * String(experiment.condition) * "_files"

destpath = "results/propositional/$(experiment.scale)"
:mfcc in experiment.featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in experiment.featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in experiment.featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
)

collect_rules!(rules_dict, interesting_rules, experiment)

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X25sZmlsZQ==.jl:59


3-element Vector{DisplayRule}:
 DisplayRule(▣ (mean(skwns) < 17.49449531625394) ∧ (mean(mel4=599Hz) ≥ -5.1867969051139955) ∧ (std(mel1=357Hz) ≥ 0.47553865726223216) ∧ (3ac(mel3=504Hz) < 0.14583333333333337) ∧ (qnt(mel14=3366Hz) ≥ 2.0950811543657704)  ↣  URTI
, "(mean(skwns) < 17.49) ∧ (mean(mel4=599Hz) ≥ -5.19) ∧ (std(mel1=357Hz) ≥ 0.48) ∧ (3ac(mel3=504Hz) < 0.15) ∧ (qnt(mel14=3366Hz) ≥ 2.10)", "URTI", (ninstances = 86, ncovered = 10, coverage = 0.12, confidence = 0.9, lift = 1.94, natoms = 5), (type = :propositional, condition = :URTI, scale = :semitones, featset = ()))
 DisplayRule(▣ (mean(skwns) ≥ 17.49449531625394) ∧ (min(mel1=357Hz) < -3.1725468930294074) ∧ (mean(slope) < -0.01586484705879295) ∧ (qnt(cntrd) < 2.0525548285632675)  ↣  Healthy
, "(mean(skwns) ≥ 17.49) ∧ (min(mel1=357Hz) < -3.17) ∧ (mean(slope) < -0.02) ∧ (qnt(cntrd) < 2.05)", "Healthy", (ninstances = 86, ncovered = 10, coverage = 0.12, confidence = 0.6, lift = 1.12, natoms = 4), (type = :propositional, condition = :U

### URTI
scale: semitones with mfcc

In [11]:
experiment = (
    type = :propositional,
    # type = :modal,

    # condition = :Pneumonia,
    # condition = :Bronchiectasis,
    # condition = :COPD,
    condition = :URTI,
    # condition = :Bronchiolitis,

    scale = :semitones,
    # scale = :mel_htk,

    # featset = (),
    featset = (:mfcc,),
)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = experiment.scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = experiment.scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = experiment.scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in experiment.featset ? false : true,
    audio_norm = true,
)

@assert experiment.condition in avail_exp "Unknown type of experiment: $(experiment.condition)."

ds_path = "/datasets/respiratory_Healthy_" * String(experiment.condition)
filename = "/datasets/itadata2024_" * String(experiment.condition) * "_files"

destpath = "results/propositional/$(experiment.scale)"
:mfcc in experiment.featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in experiment.featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in experiment.featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
)

collect_rules!(rules_dict, interesting_rules, experiment)

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X30sZmlsZQ==.jl:59


5-element Vector{DisplayRule}:
 DisplayRule(▣ (mean(skwns) < 17.49449531625394) ∧ (mean(mel4=599Hz) ≥ -5.1867969051139955) ∧ (std(mel1=357Hz) ≥ 0.47553865726223216) ∧ (3ac(mel3=504Hz) < 0.14583333333333337) ∧ (qnt(mel14=3366Hz) ≥ 2.0950811543657704)  ↣  URTI
, "(mean(skwns) < 17.49) ∧ (mean(mel4=599Hz) ≥ -5.19) ∧ (std(mel1=357Hz) ≥ 0.48) ∧ (3ac(mel3=504Hz) < 0.15) ∧ (qnt(mel14=3366Hz) ≥ 2.10)", "URTI", (ninstances = 86, ncovered = 10, coverage = 0.12, confidence = 0.9, lift = 1.94, natoms = 5), (type = :propositional, condition = :URTI, scale = :semitones, featset = ()))
 DisplayRule(▣ (mean(skwns) ≥ 17.49449531625394) ∧ (min(mel1=357Hz) < -3.1725468930294074) ∧ (mean(slope) < -0.01586484705879295) ∧ (qnt(cntrd) < 2.0525548285632675)  ↣  Healthy
, "(mean(skwns) ≥ 17.49) ∧ (min(mel1=357Hz) < -3.17) ∧ (mean(slope) < -0.02) ∧ (qnt(cntrd) < 2.05)", "Healthy", (ninstances = 86, ncovered = 10, coverage = 0.12, confidence = 0.6, lift = 1.12, natoms = 4), (type = :propositional, condition = :U

### Bronchiolitis
scale: semitones without mfcc

In [12]:
experiment = (
    type = :propositional,
    # type = :modal,

    # condition = :Pneumonia,
    # condition = :Bronchiectasis,
    # condition = :COPD,
    # condition = :URTI,
    condition = :Bronchiolitis,

    scale = :semitones,
    # scale = :mel_htk,

    featset = (),
    # featset = (:mfcc,),
)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = experiment.scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = experiment.scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = experiment.scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in experiment.featset ? false : true,
    audio_norm = true,
)

@assert experiment.condition in avail_exp "Unknown type of experiment: $(experiment.condition)."

ds_path = "/datasets/respiratory_Healthy_" * String(experiment.condition)
filename = "/datasets/itadata2024_" * String(experiment.condition) * "_files"

destpath = "results/propositional/$(experiment.scale)"
:mfcc in experiment.featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in experiment.featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in experiment.featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
)

collect_rules!(rules_dict, interesting_rules, experiment)

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X32sZmlsZQ==.jl:59


3-element Vector{DisplayRule}:
 DisplayRule(▣ (mean(kurts) < 5761.282108261717) ∧ (med(mel6=845Hz) < -3.253200202374699) ∧ (mean(entrp) < 0.2113400906078905) ∧ (max(mel1=357Hz) ≥ -2.4734158591872473) ∧ (mean(skwns) < 15.58389831545624) ∧ (std(mel3=504Hz) ≥ 0.5690149658278374) ∧ (qnt(kurts) < 2.178346906581175)  ↣  Bronchiolitis
, "(mean(kurts) < 5761.28) ∧ (med(mel6=845Hz) < -3.25) ∧ (mean(entrp) < 0.21) ∧ (max(mel1=357Hz) ≥ -2.47) ∧ (mean(skwns) < 15.58) ∧ (std(mel3=504Hz) ≥ 0.57) ∧ (qnt(kurts) < 2.18)", "Bronchiolitis", (ninstances = 56, ncovered = 6, coverage = 0.11, confidence = 1.0, lift = 2.07, natoms = 7), (type = :propositional, condition = :Bronchiolitis, scale = :semitones, featset = ()))
 DisplayRule(▣ (mean(kurts) < 5761.282108261717) ∧ (med(mel6=845Hz) < -3.253200202374699) ∧ (mean(entrp) ≥ 0.2113400906078905) ∧ (min(skwns) < 3.3512815656045105)  ↣  Bronchiolitis
, "(mean(kurts) < 5761.28) ∧ (med(mel6=845Hz) < -3.25) ∧ (mean(entrp) ≥ 0.21) ∧ (min(skwns) < 3.35)", "Bronchio

### Bronchiolitis
scale: semitones with mfcc

In [13]:
experiment = (
    type = :propositional,
    # type = :modal,

    # condition = :Pneumonia,
    # condition = :Bronchiectasis,
    # condition = :COPD,
    # condition = :URTI,
    condition = :Bronchiolitis,

    scale = :semitones,
    # scale = :mel_htk,

    # featset = (),
    featset = (:mfcc,),
)

audioparams = (
    sr = sr,
    nfft = 512,
    mel_scale = experiment.scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = experiment.scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = experiment.scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in experiment.featset ? false : true,
    audio_norm = true,
)

@assert experiment.condition in avail_exp "Unknown type of experiment: $(experiment.condition)."

ds_path = "/datasets/respiratory_Healthy_" * String(experiment.condition)
filename = "/datasets/itadata2024_" * String(experiment.condition) * "_files"

destpath = "results/propositional/$(experiment.scale)"
:mfcc in experiment.featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])m$j(mel$i=$(freq[i])Hz)\e[0m" for i in 1:audioparams.mel_nbands],
        :mfcc in experiment.featset ? ["\e[$(color_code[:red])m$j(mfcc$i)\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
        :f0 in experiment.featset ? ["\e[$(color_code[:green])m$j(f0)\e[0m"] : String[],
        "\e[$(color_code[:cyan])m$j(cntrd)\e[0m", "\e[$(color_code[:cyan])m$j(crest)\e[0m",
        "\e[$(color_code[:cyan])m$j(entrp)\e[0m", "\e[$(color_code[:cyan])m$j(flatn)\e[0m", "\e[$(color_code[:cyan])m$j(flux)\e[0m",
        "\e[$(color_code[:cyan])m$j(kurts)\e[0m", "\e[$(color_code[:cyan])m$j(rllff)\e[0m", "\e[$(color_code[:cyan])m$j(skwns)\e[0m",
        "\e[$(color_code[:cyan])m$j(decrs)\e[0m", "\e[$(color_code[:cyan])m$j(slope)\e[0m", "\e[$(color_code[:cyan])m$j(sprd)\e[0m"
    )
    for j in catch9_f
]...)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
)

collect_rules!(rules_dict, interesting_rules, experiment)

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X34sZmlsZQ==.jl:59


7-element Vector{DisplayRule}:
 DisplayRule(▣ (mean(kurts) < 5761.282108261717) ∧ (med(mel6=845Hz) < -3.253200202374699) ∧ (mean(entrp) < 0.2113400906078905) ∧ (max(mel1=357Hz) ≥ -2.4734158591872473) ∧ (mean(skwns) < 15.58389831545624) ∧ (std(mel3=504Hz) ≥ 0.5690149658278374) ∧ (qnt(kurts) < 2.178346906581175)  ↣  Bronchiolitis
, "(mean(kurts) < 5761.28) ∧ (med(mel6=845Hz) < -3.25) ∧ (mean(entrp) < 0.21) ∧ (max(mel1=357Hz) ≥ -2.47) ∧ (mean(skwns) < 15.58) ∧ (std(mel3=504Hz) ≥ 0.57) ∧ (qnt(kurts) < 2.18)", "Bronchiolitis", (ninstances = 56, ncovered = 6, coverage = 0.11, confidence = 1.0, lift = 2.07, natoms = 7), (type = :propositional, condition = :Bronchiolitis, scale = :semitones, featset = ()))
 DisplayRule(▣ (mean(kurts) < 5761.282108261717) ∧ (med(mel6=845Hz) < -3.253200202374699) ∧ (mean(entrp) ≥ 0.2113400906078905) ∧ (min(skwns) < 3.3512815656045105)  ↣  Bronchiolitis
, "(mean(kurts) < 5761.28) ∧ (med(mel6=845Hz) < -3.25) ∧ (mean(entrp) ≥ 0.21) ∧ (min(skwns) < 3.35)", "Bronchio

## modal analysis intersting rules comparision

In [14]:
function mean_longstretch1(x) Catch22.SB_BinaryStats_mean_longstretch1((x)) end
function diff_longstretch0(x) Catch22.SB_BinaryStats_diff_longstretch0((x)) end
function quantile_hh(x) Catch22.SB_MotifThree_quantile_hh((x)) end
function sumdiagcov(x) Catch22.SB_TransitionMatrix_3ac_sumdiagcov((x)) end

function histogramMode_5(x) Catch22.DN_HistogramMode_5((x)) end
function f1ecac(x) Catch22.CO_f1ecac((x)) end
function histogram_even_2_5(x) Catch22.CO_HistogramAMI_even_2_5((x)) end

function get_patched_feature(f::Base.Callable, polarity::Symbol)
    if f in [minimum, maximum, StatsBase.mean, median]
        f
    else
        @eval $(Symbol(string(f)*string(polarity)))
    end
end

features = :catch9
# features = :minmax
# features = :custom

color_code = Dict(:red => 31, :green => 32, :yellow => 33, :blue => 34, :magenta => 35, :cyan => 36);
r_select = r"\e\[\d+m(.*?)\e\[0m";

nan_guard = [:std, :mean_longstretch1, :diff_longstretch0, :quantile_hh, :sumdiagcov, :histogramMode_5, :f1ecac, :histogram_even_2_5]

for f_name in nan_guard
    @eval (function $(Symbol(string(f_name)*"+"))(channel)
        val = $(f_name)(channel)

        if isnan(val)
            SoleData.aggregator_bottom(SoleData.existential_aggregator(≥), eltype(channel))
        else
            eltype(channel)(val)
        end
    end)
    @eval (function $(Symbol(string(f_name)*"-"))(channel)
        val = $(f_name)(channel)

        if isnan(val)
            SoleData.aggregator_bottom(SoleData.existential_aggregator(≤), eltype(channel))
        else
            eltype(channel)(val)
        end
    end)
end

if features == :catch9
    metaconditions = [
        (≥, get_patched_feature(maximum, :+)),            (≤, get_patched_feature(maximum, :-)),
        (≥, get_patched_feature(minimum, :+)),            (≤, get_patched_feature(minimum, :-)),
        (≥, get_patched_feature(StatsBase.mean, :+)),     (≤, get_patched_feature(StatsBase.mean, :-)),
        (≥, get_patched_feature(median, :+)),             (≤, get_patched_feature(median, :-)),
        (≥, get_patched_feature(std, :+)),                (≤, get_patched_feature(std, :-)),
        (≥, get_patched_feature(mean_longstretch1, :+)),  (≤, get_patched_feature(mean_longstretch1, :-)),
        (≥, get_patched_feature(diff_longstretch0, :+)),  (≤, get_patched_feature(diff_longstretch0, :-)),
        (≥, get_patched_feature(quantile_hh, :+)),        (≤, get_patched_feature(quantile_hh, :-)),
        (≥, get_patched_feature(sumdiagcov, :+)),         (≤, get_patched_feature(sumdiagcov, :-)),
    ]
elseif features == :minmax
    metaconditions = [
        (≥, get_patched_feature(maximum, :+)),            (≤, get_patched_feature(maximum, :-)),
        (≥, get_patched_feature(minimum, :+)),            (≤, get_patched_feature(minimum, :-)),
    ]
elseif features == :custom
    metaconditions = [
        (≥, get_patched_feature(maximum, :+)),            (≤, get_patched_feature(maximum, :-)),
        # (≥, get_patched_feature(minimum, :+)),            (≤, get_patched_feature(minimum, :-)),
        # (≥, get_patched_feature(StatsBase.mean, :+)),     (≤, get_patched_feature(StatsBase.mean, :-)),
        # (≥, get_patched_feature(median, :+)),             (≤, get_patched_feature(median, :-)),
        (≥, get_patched_feature(std, :+)),                (≤, get_patched_feature(std, :-)),
        # (≥, get_patched_feature(mean_longstretch1, :+)),  (≤, get_patched_feature(mean_longstretch1, :-)),
        # (≥, get_patched_feature(diff_longstretch0, :+)),  (≤, get_patched_feature(diff_longstretch0, :-)),
        # (≥, get_patched_feature(quantile_hh, :+)),        (≤, get_patched_feature(quantile_hh, :-)),
        # (≥, get_patched_feature(sumdiagcov, :+)),         (≤, get_patched_feature(sumdiagcov, :-)),
        (≥, get_patched_feature(histogramMode_5, :+)),    (≤, get_patched_feature(histogramMode_5, :-)),
        (≥, get_patched_feature(f1ecac, :+)),             (≤, get_patched_feature(f1ecac, :-)),
        (≥, get_patched_feature(histogram_even_2_5, :+)), (≤, get_patched_feature(histogram_even_2_5, :-)),
    ]
else
    error("Unknown set of features: $features.")
end

18-element Vector{Tuple{Function, Function}}:
 (>=, maximum)
 (<=, maximum)
 (>=, minimum)
 (<=, minimum)
 (>=, Statistics.mean)
 (<=, Statistics.mean)
 (>=, Statistics.median)
 (<=, Statistics.median)
 (>=, var"std+")
 (<=, var"std-")
 (>=, var"mean_longstretch1+")
 (<=, var"mean_longstretch1-")
 (>=, var"diff_longstretch0+")
 (<=, var"diff_longstretch0-")
 (>=, var"quantile_hh+")
 (<=, var"quantile_hh-")
 (>=, var"sumdiagcov+")
 (<=, var"sumdiagcov-")

### Pneumonia
scale: semitones without mfcc

In [15]:
experiment = (
    # type = :propositional,
    type = :modal,

    condition = :Pneumonia,
    # condition = :Bronchiectasis,
    # condition = :COPD,
    # condition = :URTI,
    # condition = :Bronchiolitis,

    scale = :semitones,
    # scale = :mel_htk,

    featset = (),
    # featset = (:mfcc,),
)

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
@assert experiment.condition in avail_exp "Unknown type of experiment: $(experiment.condition)."

destpath = "results/modal/$(experiment.scale)"
:mfcc in experiment.featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"

sr = 8000

audioparams = (
    sr = sr,
    # nfft = 256,
    nfft = 512,
    mel_scale = experiment.scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = experiment.scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = experiment.scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in experiment.featset ? false : true,
    audio_norm = true,
)

findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment.condition), y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment.condition)
filename = "/datasets/itadata2024_" * String(experiment.condition) * "_files"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat(
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:audioparams.mel_nbands],
    :mfcc in experiment.featset ? ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
    :f0 in experiment.featset ? ["\e[$(color_code[:green])mf0\e[0m"] : String[],
    "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
)

collect_rules!(rules_dict, interesting_rules, experiment; variable_names=variable_names)

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X41sZmlsZQ==.jl:62


8-element Vector{DisplayRule}:
 DisplayRule(▣ (std(skwns) < 16.8775664684418) ∧ (std(mel10=1687Hz) < 0.5352387834210879) ∧ (min(mel8=1194Hz) < -4.203758541572919) ∧ (std(mel8=1194Hz) < 0.18671195168672658)  ↣  Pneumonia
, "(std(skwns) < 16.88) ∧ (std(mel10=1687Hz) < 0.54) ∧ (min(mel8=1194Hz) < -4.20) ∧ (std(mel8=1194Hz) < 0.19)", "Pneumonia", (ninstances = 101, ncovered = 17, coverage = 0.17, confidence = 1.0, lift = 2.2, natoms = 4), (type = :propositional, condition = :Pneumonia, scale = :semitones, featset = ()))
 DisplayRule(▣ (std(skwns) < 16.8775664684418) ∧ (std(mel10=1687Hz) < 0.5352387834210879) ∧ (min(mel8=1194Hz) < -4.203758541572919) ∧ (std(mel8=1194Hz) ≥ 0.18671195168672658) ∧ (max(mel6=845Hz) ≥ -3.5792143086384973) ∧ (std(mel6=845Hz) ≥ 0.48144848701559795) ∧ (max(decrs) < 0.9335043307891185)  ↣  Pneumonia
, "(std(skwns) < 16.88) ∧ (std(mel10=1687Hz) < 0.54) ∧ (min(mel8=1194Hz) < -4.20) ∧ (std(mel8=1194Hz) ≥ 0.19) ∧ (max(mel6=845Hz) ≥ -3.58) ∧ (std(mel6=845Hz) ≥ 0.48) ∧ (m

### Pneumonia
scale: semitones with mfcc

In [16]:
experiment = (
    # type = :propositional,
    type = :modal,

    condition = :Pneumonia,
    # condition = :Bronchiectasis,
    # condition = :COPD,
    # condition = :URTI,
    # condition = :Bronchiolitis,

    scale = :semitones,
    # scale = :mel_htk,

    # featset = (),
    featset = (:mfcc,),
)

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
@assert experiment.condition in avail_exp "Unknown type of experiment: $(experiment.condition)."

destpath = "results/modal/$(experiment.scale)"
:mfcc in experiment.featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"

sr = 8000

audioparams = (
    sr = sr,
    # nfft = 256,
    nfft = 512,
    mel_scale = experiment.scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = experiment.scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = experiment.scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in experiment.featset ? false : true,
    audio_norm = true,
)

findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment.condition), y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment.condition)
filename = "/datasets/itadata2024_" * String(experiment.condition) * "_files"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat(
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:audioparams.mel_nbands],
    :mfcc in experiment.featset ? ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
    :f0 in experiment.featset ? ["\e[$(color_code[:green])mf0\e[0m"] : String[],
    "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
)

collect_rules!(rules_dict, interesting_rules, experiment; variable_names=variable_names)

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X43sZmlsZQ==.jl:62


11-element Vector{DisplayRule}:
 DisplayRule(▣ (std(skwns) < 16.8775664684418) ∧ (std(mel10=1687Hz) < 0.5352387834210879) ∧ (min(mel8=1194Hz) < -4.203758541572919) ∧ (std(mel8=1194Hz) < 0.18671195168672658)  ↣  Pneumonia
, "(std(skwns) < 16.88) ∧ (std(mel10=1687Hz) < 0.54) ∧ (min(mel8=1194Hz) < -4.20) ∧ (std(mel8=1194Hz) < 0.19)", "Pneumonia", (ninstances = 101, ncovered = 17, coverage = 0.17, confidence = 1.0, lift = 2.2, natoms = 4), (type = :propositional, condition = :Pneumonia, scale = :semitones, featset = ()))
 DisplayRule(▣ (std(skwns) < 16.8775664684418) ∧ (std(mel10=1687Hz) < 0.5352387834210879) ∧ (min(mel8=1194Hz) < -4.203758541572919) ∧ (std(mel8=1194Hz) ≥ 0.18671195168672658) ∧ (max(mel6=845Hz) ≥ -3.5792143086384973) ∧ (std(mel6=845Hz) ≥ 0.48144848701559795) ∧ (max(decrs) < 0.9335043307891185)  ↣  Pneumonia
, "(std(skwns) < 16.88) ∧ (std(mel10=1687Hz) < 0.54) ∧ (min(mel8=1194Hz) < -4.20) ∧ (std(mel8=1194Hz) ≥ 0.19) ∧ (max(mel6=845Hz) ≥ -3.58) ∧ (std(mel6=845Hz) ≥ 0.48) ∧ (

### Bronchiectasis
scale: semitones without mfcc

In [17]:
experiment = (
    # type = :propositional,
    type = :modal,

    # condition = :Pneumonia,
    condition = :Bronchiectasis,
    # condition = :COPD,
    # condition = :URTI,
    # condition = :Bronchiolitis,

    scale = :semitones,
    # scale = :mel_htk,

    featset = (),
    # featset = (:mfcc,),
)

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
@assert experiment.condition in avail_exp "Unknown type of experiment: $(experiment.condition)."

destpath = "results/modal/$(experiment.scale)"
:mfcc in experiment.featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"

sr = 8000

audioparams = (
    sr = sr,
    # nfft = 256,
    nfft = 512,
    mel_scale = experiment.scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = experiment.scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = experiment.scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in experiment.featset ? false : true,
    audio_norm = true,
)

findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment.condition), y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment.condition)
filename = "/datasets/itadata2024_" * String(experiment.condition) * "_files"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat(
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:audioparams.mel_nbands],
    :mfcc in experiment.featset ? ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
    :f0 in experiment.featset ? ["\e[$(color_code[:green])mf0\e[0m"] : String[],
    "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
)

collect_rules!(rules_dict, interesting_rules, experiment; variable_names=variable_names)

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X45sZmlsZQ==.jl:62


8-element Vector{DisplayRule}:
 DisplayRule(▣ std(decrs) < 2.7746655481160873  ↣  Bronchiectasis
, "std(decrs) < 2.77", "Bronchiectasis", (ninstances = 36, ncovered = 18, coverage = 0.5, confidence = 1.0, lift = 2.0, natoms = 1), (type = :propositional, condition = :Bronchiectasis, scale = :semitones, featset = ()))
 DisplayRule(▣ std(decrs) ≥ 2.7746655481160873  ↣  Healthy
, "std(decrs) ≥ 2.77", "Healthy", (ninstances = 36, ncovered = 18, coverage = 0.5, confidence = 1.0, lift = 2.0, natoms = 1), (type = :propositional, condition = :Bronchiectasis, scale = :semitones, featset = ()))
 DisplayRule(▣ mean(decrs) < -1.4909694112089293  ↣  Healthy
, "mean(decrs) < -1.49", "Healthy", (ninstances = 36, ncovered = 22, coverage = 0.61, confidence = 1.0, lift = 1.64, natoms = 1), (type = :propositional, condition = :Bronchiectasis, scale = :semitones, featset = (:mfcc,)))
 DisplayRule(▣ mean(decrs) ≥ -1.4909694112089293  ↣  Bronchiectasis
, "mean(decrs) ≥ -1.49", "Bronchiectasis", (ninstances =

### Bronchiectasis
scale: semitones with mfcc

In [18]:
experiment = (
    # type = :propositional,
    type = :modal,

    # condition = :Pneumonia,
    condition = :Bronchiectasis,
    # condition = :COPD,
    # condition = :URTI,
    # condition = :Bronchiolitis,

    scale = :semitones,
    # scale = :mel_htk,

    # featset = (),
    featset = (:mfcc,),
)

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
@assert experiment.condition in avail_exp "Unknown type of experiment: $(experiment.condition)."

destpath = "results/modal/$(experiment.scale)"
:mfcc in experiment.featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"

sr = 8000

audioparams = (
    sr = sr,
    # nfft = 256,
    nfft = 512,
    mel_scale = experiment.scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = experiment.scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = experiment.scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in experiment.featset ? false : true,
    audio_norm = true,
)

findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment.condition), y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment.condition)
filename = "/datasets/itadata2024_" * String(experiment.condition) * "_files"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat(
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:audioparams.mel_nbands],
    :mfcc in experiment.featset ? ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
    :f0 in experiment.featset ? ["\e[$(color_code[:green])mf0\e[0m"] : String[],
    "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
)

collect_rules!(rules_dict, interesting_rules, experiment; variable_names=variable_names)

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X50sZmlsZQ==.jl:62


11-element Vector{DisplayRule}:
 DisplayRule(▣ std(decrs) < 2.7746655481160873  ↣  Bronchiectasis
, "std(decrs) < 2.77", "Bronchiectasis", (ninstances = 36, ncovered = 18, coverage = 0.5, confidence = 1.0, lift = 2.0, natoms = 1), (type = :propositional, condition = :Bronchiectasis, scale = :semitones, featset = ()))
 DisplayRule(▣ std(decrs) ≥ 2.7746655481160873  ↣  Healthy
, "std(decrs) ≥ 2.77", "Healthy", (ninstances = 36, ncovered = 18, coverage = 0.5, confidence = 1.0, lift = 2.0, natoms = 1), (type = :propositional, condition = :Bronchiectasis, scale = :semitones, featset = ()))
 DisplayRule(▣ mean(decrs) < -1.4909694112089293  ↣  Healthy
, "mean(decrs) < -1.49", "Healthy", (ninstances = 36, ncovered = 22, coverage = 0.61, confidence = 1.0, lift = 1.64, natoms = 1), (type = :propositional, condition = :Bronchiectasis, scale = :semitones, featset = (:mfcc,)))
 DisplayRule(▣ mean(decrs) ≥ -1.4909694112089293  ↣  Bronchiectasis
, "mean(decrs) ≥ -1.49", "Bronchiectasis", (ninstances 

### COPD
scale: semitones without mfcc

In [19]:
experiment = (
    # type = :propositional,
    type = :modal,

    # condition = :Pneumonia,
    # condition = :Bronchiectasis,
    condition = :COPD,
    # condition = :URTI,
    # condition = :Bronchiolitis,

    scale = :semitones,
    # scale = :mel_htk,

    featset = (),
    # featset = (:mfcc,),
)

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
@assert experiment.condition in avail_exp "Unknown type of experiment: $(experiment.condition)."

destpath = "results/modal/$(experiment.scale)"
:mfcc in experiment.featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"

sr = 8000

audioparams = (
    sr = sr,
    # nfft = 256,
    nfft = 512,
    mel_scale = experiment.scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = experiment.scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = experiment.scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in experiment.featset ? false : true,
    audio_norm = true,
)

findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment.condition), y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment.condition)
filename = "/datasets/itadata2024_" * String(experiment.condition) * "_files"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat(
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:audioparams.mel_nbands],
    :mfcc in experiment.featset ? ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
    :f0 in experiment.featset ? ["\e[$(color_code[:green])mf0\e[0m"] : String[],
    "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
)

collect_rules!(rules_dict, interesting_rules, experiment; variable_names=variable_names)

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X52sZmlsZQ==.jl:62


10-element Vector{DisplayRule}:
 DisplayRule(▣ (med(entrp) < 0.20939694020224994) ∧ (min(mel12=2383Hz) ≥ -8.020823114106859) ∧ (std(mel10=1687Hz) ≥ 0.14575752636971573) ∧ (mean(slope) ≥ -0.03440939539928442)  ↣  Healthy
, "(med(entrp) < 0.21) ∧ (min(mel12=2383Hz) ≥ -8.02) ∧ (std(mel10=1687Hz) ≥ 0.15) ∧ (mean(slope) ≥ -0.03)", "Healthy", (ninstances = 110, ncovered = 26, coverage = 0.24, confidence = 0.96, lift = 1.86, natoms = 4), (type = :propositional, condition = :COPD, scale = :semitones, featset = ()))
 DisplayRule(▣ (med(entrp) ≥ 0.22865762667347816) ∧ (min(mel14=3366Hz) ≥ -5.871457993702041)  ↣  COPD
, "(med(entrp) ≥ 0.23) ∧ (min(mel14=3366Hz) ≥ -5.87)", "COPD", (ninstances = 110, ncovered = 30, coverage = 0.27, confidence = 0.97, lift = 2.01, natoms = 2), (type = :propositional, condition = :COPD, scale = :semitones, featset = ()))
 DisplayRule(▣ (mean(crest) < 120.66469656425167) ∧ (qnt(mel6=845Hz) ≥ 1.7341708652861298)  ↣  COPD
, "(mean(crest) < 120.66) ∧ (qnt(mel6=845Hz) ≥ 1

### COPD
scale: semitones with mfcc

In [20]:
experiment = (
    # type = :propositional,
    type = :modal,

    # condition = :Pneumonia,
    # condition = :Bronchiectasis,
    condition = :COPD,
    # condition = :URTI,
    # condition = :Bronchiolitis,

    scale = :semitones,
    # scale = :mel_htk,

    # featset = (),
    featset = (:mfcc,),
)

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
@assert experiment.condition in avail_exp "Unknown type of experiment: $(experiment.condition)."

destpath = "results/modal/$(experiment.scale)"
:mfcc in experiment.featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"

sr = 8000

audioparams = (
    sr = sr,
    # nfft = 256,
    nfft = 512,
    mel_scale = experiment.scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = experiment.scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = experiment.scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in experiment.featset ? false : true,
    audio_norm = true,
)

findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment.condition), y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment.condition)
filename = "/datasets/itadata2024_" * String(experiment.condition) * "_files"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat(
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:audioparams.mel_nbands],
    :mfcc in experiment.featset ? ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
    :f0 in experiment.featset ? ["\e[$(color_code[:green])mf0\e[0m"] : String[],
    "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
)

collect_rules!(rules_dict, interesting_rules, experiment; variable_names=variable_names)

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X54sZmlsZQ==.jl:62


12-element Vector{DisplayRule}:
 DisplayRule(▣ (med(entrp) < 0.20939694020224994) ∧ (min(mel12=2383Hz) ≥ -8.020823114106859) ∧ (std(mel10=1687Hz) ≥ 0.14575752636971573) ∧ (mean(slope) ≥ -0.03440939539928442)  ↣  Healthy
, "(med(entrp) < 0.21) ∧ (min(mel12=2383Hz) ≥ -8.02) ∧ (std(mel10=1687Hz) ≥ 0.15) ∧ (mean(slope) ≥ -0.03)", "Healthy", (ninstances = 110, ncovered = 26, coverage = 0.24, confidence = 0.96, lift = 1.86, natoms = 4), (type = :propositional, condition = :COPD, scale = :semitones, featset = ()))
 DisplayRule(▣ (med(entrp) ≥ 0.22865762667347816) ∧ (min(mel14=3366Hz) ≥ -5.871457993702041)  ↣  COPD
, "(med(entrp) ≥ 0.23) ∧ (min(mel14=3366Hz) ≥ -5.87)", "COPD", (ninstances = 110, ncovered = 30, coverage = 0.27, confidence = 0.97, lift = 2.01, natoms = 2), (type = :propositional, condition = :COPD, scale = :semitones, featset = ()))
 DisplayRule(▣ (mean(crest) < 120.66469656425167) ∧ (qnt(mel6=845Hz) ≥ 1.7341708652861298)  ↣  COPD
, "(mean(crest) < 120.66) ∧ (qnt(mel6=845Hz) ≥ 1

### URTI
scale: semitones without mfcc

In [21]:
experiment = (
    # type = :propositional,
    type = :modal,

    # condition = :Pneumonia,
    # condition = :Bronchiectasis,
    # condition = :COPD,
    condition = :URTI,
    # condition = :Bronchiolitis,

    scale = :semitones,
    # scale = :mel_htk,

    featset = (),
    # featset = (:mfcc,),
)

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
@assert experiment.condition in avail_exp "Unknown type of experiment: $(experiment.condition)."

destpath = "results/modal/$(experiment.scale)"
:mfcc in experiment.featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"

sr = 8000

audioparams = (
    sr = sr,
    # nfft = 256,
    nfft = 512,
    mel_scale = experiment.scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = experiment.scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = experiment.scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in experiment.featset ? false : true,
    audio_norm = true,
)

findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment.condition), y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment.condition)
filename = "/datasets/itadata2024_" * String(experiment.condition) * "_files"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat(
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:audioparams.mel_nbands],
    :mfcc in experiment.featset ? ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
    :f0 in experiment.featset ? ["\e[$(color_code[:green])mf0\e[0m"] : String[],
    "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
)

collect_rules!(rules_dict, interesting_rules, experiment; variable_names=variable_names)

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X56sZmlsZQ==.jl:62


7-element Vector{DisplayRule}:
 DisplayRule(▣ (mean(skwns) < 17.49449531625394) ∧ (mean(mel4=599Hz) ≥ -5.1867969051139955) ∧ (std(mel1=357Hz) ≥ 0.47553865726223216) ∧ (3ac(mel3=504Hz) < 0.14583333333333337) ∧ (qnt(mel14=3366Hz) ≥ 2.0950811543657704)  ↣  URTI
, "(mean(skwns) < 17.49) ∧ (mean(mel4=599Hz) ≥ -5.19) ∧ (std(mel1=357Hz) ≥ 0.48) ∧ (3ac(mel3=504Hz) < 0.15) ∧ (qnt(mel14=3366Hz) ≥ 2.10)", "URTI", (ninstances = 86, ncovered = 10, coverage = 0.12, confidence = 0.9, lift = 1.94, natoms = 5), (type = :propositional, condition = :URTI, scale = :semitones, featset = ()))
 DisplayRule(▣ (mean(skwns) ≥ 17.49449531625394) ∧ (min(mel1=357Hz) < -3.1725468930294074) ∧ (mean(slope) < -0.01586484705879295) ∧ (qnt(cntrd) < 2.0525548285632675)  ↣  Healthy
, "(mean(skwns) ≥ 17.49) ∧ (min(mel1=357Hz) < -3.17) ∧ (mean(slope) < -0.02) ∧ (qnt(cntrd) < 2.05)", "Healthy", (ninstances = 86, ncovered = 10, coverage = 0.12, confidence = 0.6, lift = 1.12, natoms = 4), (type = :propositional, condition = :U

### URTI
scale: semitones with mfcc

In [22]:
experiment = (
    # type = :propositional,
    type = :modal,

    # condition = :Pneumonia,
    # condition = :Bronchiectasis,
    # condition = :COPD,
    condition = :URTI,
    # condition = :Bronchiolitis,

    scale = :semitones,
    # scale = :mel_htk,

    # featset = (),
    featset = (:mfcc,),
)

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
@assert experiment.condition in avail_exp "Unknown type of experiment: $(experiment.condition)."

destpath = "results/modal/$(experiment.scale)"
:mfcc in experiment.featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"

sr = 8000

audioparams = (
    sr = sr,
    # nfft = 256,
    nfft = 512,
    mel_scale = experiment.scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = experiment.scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = experiment.scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in experiment.featset ? false : true,
    audio_norm = true,
)

findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment.condition), y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment.condition)
filename = "/datasets/itadata2024_" * String(experiment.condition) * "_files"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat(
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:audioparams.mel_nbands],
    :mfcc in experiment.featset ? ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
    :f0 in experiment.featset ? ["\e[$(color_code[:green])mf0\e[0m"] : String[],
    "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
)

collect_rules!(rules_dict, interesting_rules, experiment; variable_names=variable_names)

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X61sZmlsZQ==.jl:62


10-element Vector{DisplayRule}:
 DisplayRule(▣ (mean(skwns) < 17.49449531625394) ∧ (mean(mel4=599Hz) ≥ -5.1867969051139955) ∧ (std(mel1=357Hz) ≥ 0.47553865726223216) ∧ (3ac(mel3=504Hz) < 0.14583333333333337) ∧ (qnt(mel14=3366Hz) ≥ 2.0950811543657704)  ↣  URTI
, "(mean(skwns) < 17.49) ∧ (mean(mel4=599Hz) ≥ -5.19) ∧ (std(mel1=357Hz) ≥ 0.48) ∧ (3ac(mel3=504Hz) < 0.15) ∧ (qnt(mel14=3366Hz) ≥ 2.10)", "URTI", (ninstances = 86, ncovered = 10, coverage = 0.12, confidence = 0.9, lift = 1.94, natoms = 5), (type = :propositional, condition = :URTI, scale = :semitones, featset = ()))
 DisplayRule(▣ (mean(skwns) ≥ 17.49449531625394) ∧ (min(mel1=357Hz) < -3.1725468930294074) ∧ (mean(slope) < -0.01586484705879295) ∧ (qnt(cntrd) < 2.0525548285632675)  ↣  Healthy
, "(mean(skwns) ≥ 17.49) ∧ (min(mel1=357Hz) < -3.17) ∧ (mean(slope) < -0.02) ∧ (qnt(cntrd) < 2.05)", "Healthy", (ninstances = 86, ncovered = 10, coverage = 0.12, confidence = 0.6, lift = 1.12, natoms = 4), (type = :propositional, condition = :

### Bronchiolitis
scale: semitones without mfcc

In [23]:
experiment = (
    # type = :propositional,
    type = :modal,

    # condition = :Pneumonia,
    # condition = :Bronchiectasis,
    # condition = :COPD,
    # condition = :URTI,
    condition = :Bronchiolitis,

    scale = :semitones,
    # scale = :mel_htk,

    featset = (),
    # featset = (:mfcc,),
)

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
@assert experiment.condition in avail_exp "Unknown type of experiment: $(experiment.condition)."

destpath = "results/modal/$(experiment.scale)"
:mfcc in experiment.featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"

sr = 8000

audioparams = (
    sr = sr,
    # nfft = 256,
    nfft = 512,
    mel_scale = experiment.scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = experiment.scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = experiment.scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in experiment.featset ? false : true,
    audio_norm = true,
)

findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment.condition), y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment.condition)
filename = "/datasets/itadata2024_" * String(experiment.condition) * "_files"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat(
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:audioparams.mel_nbands],
    :mfcc in experiment.featset ? ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
    :f0 in experiment.featset ? ["\e[$(color_code[:green])mf0\e[0m"] : String[],
    "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
)

collect_rules!(rules_dict, interesting_rules, experiment; variable_names=variable_names)

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X63sZmlsZQ==.jl:62


9-element Vector{DisplayRule}:
 DisplayRule(▣ (mean(kurts) < 5761.282108261717) ∧ (med(mel6=845Hz) < -3.253200202374699) ∧ (mean(entrp) < 0.2113400906078905) ∧ (max(mel1=357Hz) ≥ -2.4734158591872473) ∧ (mean(skwns) < 15.58389831545624) ∧ (std(mel3=504Hz) ≥ 0.5690149658278374) ∧ (qnt(kurts) < 2.178346906581175)  ↣  Bronchiolitis
, "(mean(kurts) < 5761.28) ∧ (med(mel6=845Hz) < -3.25) ∧ (mean(entrp) < 0.21) ∧ (max(mel1=357Hz) ≥ -2.47) ∧ (mean(skwns) < 15.58) ∧ (std(mel3=504Hz) ≥ 0.57) ∧ (qnt(kurts) < 2.18)", "Bronchiolitis", (ninstances = 56, ncovered = 6, coverage = 0.11, confidence = 1.0, lift = 2.07, natoms = 7), (type = :propositional, condition = :Bronchiolitis, scale = :semitones, featset = ()))
 DisplayRule(▣ (mean(kurts) < 5761.282108261717) ∧ (med(mel6=845Hz) < -3.253200202374699) ∧ (mean(entrp) ≥ 0.2113400906078905) ∧ (min(skwns) < 3.3512815656045105)  ↣  Bronchiolitis
, "(mean(kurts) < 5761.28) ∧ (med(mel6=845Hz) < -3.25) ∧ (mean(entrp) ≥ 0.21) ∧ (min(skwns) < 3.35)", "Bronchio

### Bronchiolitis
scale: semitones with mfcc

In [24]:
experiment = (
    # type = :propositional,
    type = :modal,

    # condition = :Pneumonia,
    # condition = :Bronchiectasis,
    # condition = :COPD,
    # condition = :URTI,
    condition = :Bronchiolitis,

    scale = :semitones,
    # scale = :mel_htk,

    # featset = (),
    featset = (:mfcc,),
)

avail_exp = [:Pneumonia, :Bronchiectasis, :COPD, :URTI, :Bronchiolitis]
@assert experiment.condition in avail_exp "Unknown type of experiment: $(experiment.condition)."

destpath = "results/modal/$(experiment.scale)"
:mfcc in experiment.featset ? destpath *= "_mfcc/" : destpath *= "/"
jld2file = destpath * "/itadata2024_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"
dsfile = destpath * "/ds_test_" * String(experiment.condition) * "_" * String(experiment.scale) * ".jld2"

sr = 8000

audioparams = (
    sr = sr,
    # nfft = 256,
    nfft = 512,
    mel_scale = experiment.scale, # :mel_htk, :mel_slaney, :erb, :bark, :semitones, :tuned_semitones
    mel_nbands = experiment.scale == :semitones ? 14 : 26,
    mfcc_ncoeffs = experiment.scale == :semitones ? 7 : 13,
    mel_freqrange = (300, round(Int, sr / 2)),
    mel_dbscale = :mfcc in experiment.featset ? false : true,
    audio_norm = true,
)

findhealthy = y -> findall(x -> x == "Healthy", y)
findsick = y -> findall(x -> x == String(experiment.condition), y)
ds_path = "/datasets/respiratory_Healthy_" * String(experiment.condition)
filename = "/datasets/itadata2024_" * String(experiment.condition) * "_files"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

freq = round.(Int, afe(x[1, :audio]; featset=(:get_only_freqs), audioparams...))

variable_names = vcat(
    ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz\e[0m" for i in 1:audioparams.mel_nbands],
    :mfcc in experiment.featset ? ["\e[$(color_code[:red])mmfcc$i\e[0m" for i in 1:audioparams.mfcc_ncoeffs] : String[],
    :f0 in experiment.featset ? ["\e[$(color_code[:green])mf0\e[0m"] : String[],
    "\e[$(color_code[:cyan])mcntrd\e[0m", "\e[$(color_code[:cyan])mcrest\e[0m",
    "\e[$(color_code[:cyan])mentrp\e[0m", "\e[$(color_code[:cyan])mflatn\e[0m", "\e[$(color_code[:cyan])mflux\e[0m",
    "\e[$(color_code[:cyan])mkurts\e[0m", "\e[$(color_code[:cyan])mrllff\e[0m", "\e[$(color_code[:cyan])mskwns\e[0m",
    "\e[$(color_code[:cyan])mdecrs\e[0m", "\e[$(color_code[:cyan])mslope\e[0m", "\e[$(color_code[:cyan])msprd\e[0m"
)

@info("Load dataset...")
d = jldopen(dsfile)
X_test = d["X_test"]
y_test = d["y_test"]
close(d)
d = jldopen(jld2file)
sole_dt = d["sole_dt"]
close(d)

interesting_rules = listrules(sole_dt,
	min_lift = 1.0,
	# min_lift = 2.0,
	min_ninstances = 0,
	min_coverage = 0.10,
	normalize = true,
)

collect_rules!(rules_dict, interesting_rules, experiment; variable_names=variable_names)

┌ Info: Load dataset...
└ @ Main /home/paso/Documents/Aclai/audio-rules2024/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X65sZmlsZQ==.jl:62


11-element Vector{DisplayRule}:
 DisplayRule(▣ (mean(kurts) < 5761.282108261717) ∧ (med(mel6=845Hz) < -3.253200202374699) ∧ (mean(entrp) < 0.2113400906078905) ∧ (max(mel1=357Hz) ≥ -2.4734158591872473) ∧ (mean(skwns) < 15.58389831545624) ∧ (std(mel3=504Hz) ≥ 0.5690149658278374) ∧ (qnt(kurts) < 2.178346906581175)  ↣  Bronchiolitis
, "(mean(kurts) < 5761.28) ∧ (med(mel6=845Hz) < -3.25) ∧ (mean(entrp) < 0.21) ∧ (max(mel1=357Hz) ≥ -2.47) ∧ (mean(skwns) < 15.58) ∧ (std(mel3=504Hz) ≥ 0.57) ∧ (qnt(kurts) < 2.18)", "Bronchiolitis", (ninstances = 56, ncovered = 6, coverage = 0.11, confidence = 1.0, lift = 2.07, natoms = 7), (type = :propositional, condition = :Bronchiolitis, scale = :semitones, featset = ()))
 DisplayRule(▣ (mean(kurts) < 5761.282108261717) ∧ (med(mel6=845Hz) < -3.253200202374699) ∧ (mean(entrp) ≥ 0.2113400906078905) ∧ (min(skwns) < 3.3512815656045105)  ↣  Bronchiolitis
, "(mean(kurts) < 5761.28) ∧ (med(mel6=845Hz) < -3.25) ∧ (mean(entrp) ≥ 0.21) ∧ (min(skwns) < 3.35)", "Bronchi

## Save rules to csv

In [30]:
for (key, value) in rules_dict
    csvname = "interesting_rules_" * string(key)

    if !isempty(value)
        X = DataFrame(
            rule=String[], consequent=String[], 
            coverage=Float64[], confidence=Float64[], lift=Float64[], natoms=Int64[],
            type=String[], condition=String[], scale=String[], featset=String[])
        for rule in value
            push!(X, vcat(rule.printrule, rule.consequent,
            rule.metrics.coverage, rule.metrics.confidence, rule.metrics.lift, rule.metrics.natoms,
            string(rule.experiment.type), string(rule.experiment.condition), string(rule.experiment.scale), string(rule.experiment.featset)))
        end
        CSV.write(string(csvname, ".csv"), X)
    else
        @warn"No rules found in " * string(key) * "."
    end
end